<h1>TITEL TBD</h1>

Om nuttige voorspellingen te doen, moet eerst bepaald worden wat voorspeld moet worden. Dit document in bedoelt om later in het hoofd-document in integreren, maar is voor nu stand-alone, en dekt de Business-Understanding, Data-Understanding en (gedeeltelijk) Data-Preparation voor het target variabele af.

<h2>Target variabele.</h2>

<h3>Business understanding</h3>

Door de interviews met onze business-expert hebben begrepen dat het voorspellen van momemt van 'functieherstel' toegevoegde waarde voor ProRail zou hebben. Dit is het moment dat een traject weer klaar is om treinverkeer toe te staan. Via ProRail hebben wij dit diagram aangeleverd gekregen, wat inzicht geeft in het bedrijfsprocess:

![title](images/badkuip.png)
De X-as beslaat tijd, en de Y-as hoeveelheid treinverkeer (beiden non-proportioneel). Dit model zal naar gerefereerd worden als 'badkuip model'.

Functieherstel staat niet expliciet benoemd in dit model, maar vind plaats in de buurt van het "Storing-Eind" moment, en wanneer de lijn weer omhoog begint te bewegen op de Y-as.

ProRail baat bij een goede voorspelling wat betreft moment van functieherstel, zodat ze kunnen zorgen dat het opstarten van de dienstregeling zo goed mogelijk hierop kan aansluiten.

Uit de interviews blijkt dat de gegevens die waarschijnlijk tot een goede prognose leiden waarschijnlijk pas beschikbaar zijn wanneer een aannemer ter plekke is, in het badkuipmodel benoemd als "oplosteam aanwezig". Als het goed is zijn zowel het tijdstip van aankomst van de monteur, als het tijdstip van functieherstel aanwezig in de dataset, hoewel dit later in 'Data Understanding' bevestigd moet worden.

De verstreken tijd tussen de melding van een incident, en het moment dat een oplosteam aanwezig is is sterk variabel. Hoewel dit in de toekomst wellicht ook te voorspellen zou zijn met een model, beperken wij in onze huidige aanpak tot het voorspellen van 'aankomst aannemer' tot 'functieherstel', omdat dat volgens onze huidige business-kennis het best voorspelbaar is aan de hand van de attributen van een specifieke storing.

<h3>Data Understanding.</h3>

Om de mogelijkheid van het implementeren van het targetvariabele van (tijdstip functieherstel) - (tijdstip aankomst aannemer) te onderzoeken, moet de data geïnspecteerd om te bepalen of dit ook echt realistisch is. Hiervoor onderzoeken we de eerder benoemde hypothese voor targetvariabele, en eventuele variabelen die hier concurrentie voor zouden kunnen zijn, of meer inzicht zouden kunnen geven.

We beginnen met het importeren van een aantal libraries, en de dataset:

In [ ]:
import enum

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn import dummy

In [ ]:
df = pd.read_csv("sap_storing_data_hu_project.csv")
df

Omdat de dataset redelijk groot is, kosten veel van de operaties die tijdens het verkennen van de data gebeuren, erg veel tijd. Volgens onze business-expert zou een willikeurige selectie van ongeveer 10% van de dataset alsnog representatief voor de hele dataset moeten zijn, dus voor het verkennen maken we dit meteen aan.

In [ ]:
sample = df.sample(frac=0.1, random_state=0)

Iets interessants om te onderzoeken is de relatie tussen de tijdstippen van functieherstel, storing eind, en tijdstip van melding. We maken voor de onderzoek doeleinden hier aparte variabelen van voor gemak, en kijken naar de datatypes.

In [ ]:
fh_tijdstip = sample["stm_fh_ddt"]
stor_eind = sample["stm_sap_storeind_ddt"]
stm_sap_meld_ddt = sample["stm_sap_meld_ddt"]

In [ ]:
print(f"""{fh_tijdstip.dtype}
{stor_eind.dtype}
{stm_sap_meld_ddt.dtype}""")

Al deze kolommen bestaan uit Strings. Deze moeten omgezet worden naar DateTime voor effectief rekenwerk. Hierna slaan we ze op in een apart DataFrame voor onderzoek. Uit dit dataframe verwijderen we alle rijen waar minimaal 1 NaN waarde instaat, omdat dit dataframe bedoelt is om de werking tussen de verschillende kolommen te vergelijken.

Ook definieren we een TimeDelta object voor 2 minuten, omdat het exact vergelijken van tijdstippen voor onze doelen vaak niet nuttig is, en als dingen binnen 2 minuten van elkaar vallen, dit in veel gevallen meer informatie kan geven.

In [ ]:
fh_tijdstip = pd.to_datetime(fh_tijdstip, errors="coerce")
stor_eind = pd.to_datetime(stor_eind, errors="coerce")
stm_sap_meld_ddt = pd.to_datetime(stm_sap_meld_ddt, errors="coerce")

twominutes = pd.to_timedelta(2, unit="m")

In [ ]:
#sample_de_1 = DataFrame DataExploration 1
df_de_1 = pd.DataFrame({"fh_ddt": fh_tijdstip,
                       "se_ddt": stor_eind,
                       "meld_ddt": stm_sap_meld_ddt}).dropna()

df_de_1.shape  # 1/3rd is dropped from nas

Een eerste vraag om te onderzoek is naar het verschil tussen "Storing eind" en "Functieherstel". Uit ons business-onderzoek begrijpen we dat deze waarschijnlijk de volgende betekenissen hebben:

Functieherstel: Het moment dat er theoretisch weer treinverkeer over het traject kan.
Storing eind: Wanneer het probleem opgelost is, en de herstelwerkzaamheden voltooid zijn.

Omdat het probleem vaak opgelost is, wanneer er weer treinen over het traject kunnen, zullen deze 2 momenten vaak tegelijk zijn. Dit kunnen we ook bevestigen:

In [ ]:
def percentage(numerator: float, denominator: float) -> float:
    """Calculate a percentage from a fraction.

    Args:
        numerator: fraction numerator.
        denominator: fraction denominator.

    Returns:
        Percentage between 0 and 100, as float."""
    return (numerator / denominator) * 100


n_fh_is_se = (np.abs(df_de_1["fh_ddt"] - df_de_1["se_ddt"]) < twominutes).sum()
f"Storing Eind is in {percentage(n_fh_is_se, df_de_1.shape[0])}% van de gevallen binnen 2 minuten van Functie Herstel."

In veel andere gevallen zou men verwachten dat storing eind na functieherstel is, wanneer de aannemer het spoor weer operationeel krijgt, maar daarna wel nog werk moet afmaken.

In [ ]:
n_fh_voor_se = ((df_de_1["se_ddt"] - twominutes) > df_de_1["fh_ddt"]).sum()
f"Storing eind is in {percentage(n_fh_voor_se, df_de_1.shape[0])}% van de gevallen meer dan 2 minuten na functieherstel."

Dit correspondeerd met de verwachting van Business Understanding.

Eerder hebben wij op basis van businesskennis een belovend targetvariabele vastgesteld, namelijk het verschil in tijd tussen het arriveren van de aannemer, en functieherstel. Om dit daadwerkelijk te kunnen gebruiken moet de daadwerkelijke data die hieraan correspondeerd geïnspecteerd worden.

In de data dictionairy staan specifiek een kolom voor datum en tijd (apart) benoemd voor het arriveren van de aannemer, namelijk "stm_aanntpl_dd" voor datum en "stm_aanntpl_tijd" voor tijd. In de dataset zit ook een kolom "stm_aanntpl_ddt", hoewel deze in de dictionairy als n.v.t. bestempeld is. Toch lijkt het nuttig het verschil tussen deze kolommen even te inspecteren, om te kijken welke van deze het beste is.

Ten eerste is het handig een beeld te hebben hoeveel NaN waardes er in deze kolommen zitten.

In [ ]:
n_nan_aanntpl_dd = sample["stm_aanntpl_dd"].isna().sum()
f"De datumkolom bestaat voor {percentage(n_nan_aanntpl_dd, sample.shape[0])}% uit NaN waardes."

In [ ]:
n_nan_aanntpl_tijd = sample["stm_aanntpl_tijd"].isna().sum()
f"De tijdkolom bestaat voor {percentage(n_nan_aanntpl_tijd, sample.shape[0])}% uit NaN waardes."

In [ ]:
n_nan_aanntpl_ddt = sample["stm_aanntpl_ddt"].isna().sum()
f"De gecombineerde datum en tijdkolom bestaat voor {percentage(n_nan_aanntpl_dd, sample.shape[0])}% uit NaN waardes."

De hoeveelheid NaN waardes komt overeen tussen de dd en ddt kolommen. Om te bevestigen dat deze gelijk zijn kunnen we de datum en tijd kolommen combineren naar één, en ze allemaal als DateTime objecten in een apart DataFrame zetten.

In [ ]:
ddt_format = sample["stm_aanntpl_dd"] + " " + sample["stm_aanntpl_tijd"]

df_aan = pd.DataFrame({"t": pd.to_datetime(sample["stm_aanntpl_tijd"], errors="coerce"),
                       "d": pd.to_datetime(sample["stm_aanntpl_dd"], errors="coerce"),
                       "dt": pd.to_datetime(sample["stm_aanntpl_ddt"], errors="coerce"),
                       "dt_own": pd.to_datetime(ddt_format, errors="coerce")}).dropna()


In [ ]:
f"In {percentage((df_aan['dt'] == df_aan['dt_own']).sum(), df_aan.shape[0])}% van de non-NaN gevallen is de ddt kolom gelijk aan aan de datum en tijd kolom."

Al met al kunnen we dus net zo goed de ddt_tabel in de originele dataset gebruiken als targetvariabele.

Dan nog even kijken of er idd genoeg nuttige entries zijn voor functieherstel - arrival monteur

In [ ]:
possible_target = df_de_1["fh_ddt"] - df_aan["dt"]  # Tijdstip functieherstel - tijdstip aankomst aannemer.
f"De mogelijke target bestaat voor {percentage(possible_target.isna().sum(), possible_target.shape[0])}% uit NaN-waardes."

In [ ]:
possible_target.dropna(inplace=True)
f"Wanneer deze NaN-waardes worden weggelaten hou je van de originele dataset uiteindelijk ongeveer {percentage(possible_target.shape[0], sample.shape[0])}% over die een targetvariabele hebben."

Om een nuttig targetvariabele te creeëren, zouden we uiteindelijk alle TimeDelta objecten die we gecreeërd hebben omzetten naar de hoeveelheid seconden waaruit dit tijdsinterval bestaat. Hoewel rekenen in seconden het inzicht mogelijk moeilijker maakt, verhoogt dit wel de granulariteit van het targetvariabele, waardoor deze zich meer gedraagd als continue waarde, wat de performance van het model weer ten goede komt. Om inzicht te verbeteren in performance, definieren we voor het gemak wel even een functie die een seconden-series naar minuten omzet, zodat grafieken leesbaarder gemaakt kunnen worden.

In [ ]:
class TimeUnit(enum.Enum):
    """Enum that represents a unit of measurement of time, to enable easy conversion from one unit to another.

    Values for units of measurements are measured in seconds."""
    second = 1
    minute = 60
    hour = 60 * 60
    day_night_cycle = 60 * 60 * 24


def convert_timeunit(value: pd.Series or float or int,
                     from_type: TimeUnit,
                     to_type: TimeUnit) -> float:
    """Convert an amount of time as number, or a series of those, to another.

    Not class attribute of TimeUnit class to allow for better typing.

    Args:
        value:
            a number, or a series of numbers, that represent an amount of time in a certain unit of measurement as defined in
            TimeUnit.
        from_type: TimeUnit scale of measurement of time that input is in.
        to_type: TimeUnit scale of measurement of time that output should be in.

    Returns:
        Time measurement(s) converted to new unit of measurement.
        Scalar if input was scalar, array/series of input type if input was array/series."""
    coef = from_type.value / to_type.value
    return value * coef

We creeëren voor nu een apart variabele voor het targetvariabele als seconden. Deze bevat mogelijk een aantal 0-waarden, laten we deze eerst inspecteren.

In [ ]:
de_target_secs = possible_target.dt.total_seconds()
f"Het targetvariabele bestaat voor {percentage((de_target_secs == 0).sum(), de_target_secs.shape[0])}% uit 0 seconden."

Het is wellicht interessant een beeld te hebben van outliers van ons targetvariabele:

In [ ]:
plt.boxplot(de_target_secs)

Uit deze bijna-onleesbare boxplot blijkt dat vooral boven de bovengrens, er heel veel extreme outliers zijn. Aan de onderkant zijn er ook een aantal negatieve waarden. Als we er vanuit gaan dat dit niet door foutive data word veroorzaakt, betekend dit waarschijnlijk dat er al weer treinen kunnen rijden, voordat er uberhaupt een monteur aan te pas komt. Hier moet wat aan gedaan worden.

Een standaardmethode om outliers te verwijderen, is de interkwartielafstand(<i>IQR</i>) berekenen, en alle waarnemingen 3IQR boven het 3e kwartiel, en 3IQR onder het eerste kwartiel te verwijderen. Het lijkt ons echter verstandiger in dit geval het verwijderen van outliers te baseren op business-kennis.

Om een nuttig voorspellend model te maken, is het waarschijnlijk handig om een ondergrens in functiehersteltijd te stellen vanaf het moment dat de aannemer komt. Omdat wij inschatten dat bij storingen onder de 5 minuten de aannemer zelf ook kan inschatten dat het heel kort gaat duren, en een voorspelling hier waarschijnlijk niet nuttig is, laten we deze weg uit onze dataset.

Onze business-expert heeft aangegeven alle hersteltijden boven de 6u niet interessant te vinden, dus deze waarnemingen laten we als NaN waarde in ons targetvariabele, en maken we een aparte binaire-categorie kolom voor, zodat dit eventueel in de toekomst nog gebruikt kan worden voor een apart model.

Om de verdelingen voor het eerste model te kunnen onderzoeken, halen we bij de data-exploration set nu alles onder 5 minuten en boven 6 uur eruit.



In [ ]:
de_target_secs = de_target_secs.loc[(de_target_secs < convert_timeunit(6, TimeUnit.hour, TimeUnit.second)) & (de_target_secs > convert_timeunit(5, TimeUnit.minute, TimeUnit.second))]

In [ ]:
de_target_secs.hist(bins=50)

Ons targetvariabele lijkt exponentieel verdeeld te zijn.

<h3>Data preperation.</h3>

Nu we een idee hebben van een (hopelijk) bruikbaar target variabele, kunnen de benodigde operaties op de originele dataset uitgevoerd worden, om hieruit een volledig uitgewerkt targetvariabele voor later gebruik uit te creeëren.

We beginnen door de 2 benodigde kolommen voor het target variabele naar DateTime objecten te veranderen in nieuwe kolommen.

In [ ]:
df["stm_aanntpl_ddt_as_pddt"] = pd.to_datetime(df["stm_aanntpl_ddt"], errors="coerce")
df["stm_fh_ddt_as_pddt"] = pd.to_datetime(df["stm_fh_ddt"], errors="coerce")

Hierna maken we een aparte kolom in de dataframe van het tijdstip van functieherstel, min tijdstip van arriveren van de monteur.

In [ ]:
df["fh_min_aanntpl"] = df["stm_fh_ddt_as_pddt"] - df["stm_aanntpl_ddt_as_pddt"]

We slaan van deze nieuwe kolom het verschil op als seconden in een apart variabele.

In [ ]:
target = df["fh_min_aanntpl"].dt.total_seconds()

We laten hieruit alles boven de 6 uur, en alles onder de 5 minuten weg, en slaan dit op in een nieuwe variabele.

In [ ]:
boven_5_min = target > convert_timeunit(5, TimeUnit.minute, TimeUnit.second)
onder_6_uur = target < convert_timeunit(6, TimeUnit.hour, TimeUnit.second)

target = target.loc[boven_5_min & onder_6_uur]  # Gemeten in seconden.

target.head(5)

<h3>Conversion van datetime</h3>
Hier gaan we de data uit kolom 'stm_sap_meld_ddt' bruikbaar maken voor toekomstige modellen.
Dit doen we door de datetime om te zetten naar zowel: aantal secondes vanaf middernacht & dag in het jaar

Als eerst converten we stm_sap_meld_ddt naar een pandas datetime datatype. Deze kolom krijgt de naam: 'stm_sap_meld_ddt_as_pddt'.
Op deze manier hebben we de originele data, en de omgezette data beschikbaar.

In [ ]:
df['stm_sap_meld_ddt_as_pddt'] = pd.to_datetime(df["stm_sap_meld_ddt"], errors="coerce")

Nu we een kolom hebben die in het pandas datetime datatype staat, kunnen we er gemakkelijk mee werken.

We gaan hier 2 kolomen aan df toevoegen, namelijk:
- stm_sap_meld_sec_mn:  Hierin staat het aantal secondes vanaf  00:00  tot het moment van de melding
- stm_sap_meld_day_count:  Hierin staat de hoeveelste dag in dat jaar het is, op het moment van de melding

In [ ]:
df['stm_sap_meld_sec_mn'] = (df['stm_sap_meld_ddt_as_pddt'] - df['stm_sap_meld_ddt_as_pddt'].dt.normalize()) / pd.Timedelta(seconds=1)
df['stm_sap_meld_day_count'] = df["stm_sap_meld_ddt_as_pddt"].dt.day_of_year

<h3>Modelling</h3>

Met een volledig voorbereid targetvariabele, kan er een 'BaseLine' model gemaakt worden, om een RMSE score vast te stellen die overtrefd kan worden. De meest logische strategie hiervoor is het gemiddelde gokken van alle bekende waardes. Hiervoor gebruiken we van SciKitLearn de DummyRegressor, met de 'mean' strategie.

In [ ]:
baseline = dummy.DummyRegressor(strategy="mean")
baseline_X = np.arange(target.shape[0])  # Dummy feature variabelen voor BaseLine.
baseline.fit(np.arange(target.shape[0]), target)

Van dit baseline model kunnen we vervolgens de RMSE score in seconden bepalen.

In [ ]:
baseline_rmse = metrics.mean_squared_error(target, baseline.predict(baseline_X), squared=False)
baseline_rmse

Voor inzicht is het ook wel interessant deze score even in minuten te bekijken, omdat dit toch meer inzicht geeft in de business-context.

In [ ]:
convert_timeunit(baseline_rmse, TimeUnit.second, TimeUnit.minute)

Dit model slaan we op in een DataFrame, waar later ook andere modellen in opgeslagen kunnen worden, zodat deze overzichtelijk bekeken kunnen worden.

In [ ]:
models = pd.DataFrame({"Title": ["BaseLine model with mean strategy"],
                       "Model": [baseline],
                       "RMSE": [baseline_rmse]})

models

<h2>Feature variabelen: model 1</h2>